In [ ]:
# -*- coding: utf-8 -*-
"""CS543_MP4_part1_starter_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1K75i8TjPhHM6bhr2y3uESkp2nIzyvbnW

# Google Colab setup with Google Drive folder

This notebook provides the code you need to set up Google Colab to run and import files from within a Google Drive folder.

This will allow you to upload assignment code to your Google Drive and then run the code on Google Colab machines (with free GPUs if needed).

You will need to create a folder in your Google Drive to hold your assignments and you will need to open Colaboratory within this folder before running the set up code (check the link above to see how).

Note: this use of Google Drive is optional, and you could also just manually copy the data into your colab runtime. Keep in mind, this won't be persistent though, and you will have to download your models / plots before the runtime shuts down.

# Mount Google Drive

This will allow the Colab machine to access Google Drive folders by mounting the drive on the machine. You will be asked to copy and paste an authentication code.
"""

from google.colab import drive
drive.mount('/content/gdrive/')

"""# Change directory to allow imports


As noted above, you should create a Google Drive folder to hold all your assignment files. You will need to add this code to the top of any python notebook you run to be able to import python files from your drive assignment folder (you should change the file path below to be your own assignment folder).
"""

import os
if not os.path.exists("/content/gdrive/MyDrive/CS543/MP4"):
    os.makedirs("/content/gdrive/My Drive/Colab Notebooks/CS_543_MP4")
os.chdir("/content/gdrive/MyDrive/CS543/MP4")

!ls # Check if this is your MP4 folder

"""# Set up GPU and PyTorch

First, ensure that your notebook on Colaboratory is set up to use GPU. After opening the notebook on Colaboratory, go to Edit>Notebook settings, select Python 3 under "Runtime type," select GPU under "Hardware accelerator," and save.

Next, install PyTorch:
"""

!pip3 install torch torchvision

"""Make sure that pytorch is installed and works with GPU:"""

import torch
a = torch.Tensor([1]).cuda()
print(a)

torch.cuda.is_available()

# Commented out IPython magic to ensure Python compatibility.
# imports and useful functions

from __future__ import print_function
from PIL import Image
import os
import os.path
import numpy as np
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import copy
import csv
import matplotlib
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path
from tqdm.notebook import tqdm
import sys
import torch
import torch.utils.data
import torchvision
import torchvision.transforms as transforms

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

np.random.seed(111)
torch.cuda.manual_seed_all(111)
torch.manual_seed(111)

TOTAL_CLASSES = 10
class FashionMNISTTest(torchvision.datasets.VisionDataset):
    def __init__(self, root, transform=None):
        super(FashionMNISTTest, self).__init__(root, transform=transform)

        image_filename = os.path.join(root, 'fashion_secret_test_data.npy')
        images = np.load(image_filename)

        assert len(images.shape) == 3
        assert images.shape[0] == 2000
        assert images.shape[1] == 28
        assert images.shape[2] == 28
        self.data = images

    def __getitem__(self, index):
        img = self.data[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.data)


def calculate_accuracy(dataloader, model, is_gpu):
    """ Util function to calculate val set accuracy,
    both overall and per class accuracy
    Args:
        dataloader (torch.utils.data.DataLoader): val set
        is_gpu (bool): whether to run on GPU
    Returns:
        tuple: (overall accuracy, class level accuracy)
    """
    correct = 0.
    total = 0.
    predictions = []

    class_correct = list(0. for i in range(TOTAL_CLASSES))
    class_total = list(0. for i in range(TOTAL_CLASSES))

    # Check out why .eval() is important!
    # https://discuss.pytorch.org/t/model-train-and-model-eval-vs-model-and-model-eval/5744/2
    model.eval()

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            if is_gpu:
                images = images.cuda()
                labels = labels.cuda()
            outputs = model(Variable(images))
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(list(predicted.cpu().numpy()))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    class_accuracy = 100 * np.divide(class_correct, class_total)
    return 100 * correct / total, class_accuracy


def run_secret_test(dataloader, model, is_gpu):
    predictions = []
    model.eval()

    with torch.no_grad():
      for images in dataloader:
          if is_gpu:
              images = images.cuda()
          outputs = model(Variable(images))
          predicted = torch.softmax(outputs, dim=1).cpu().numpy()
          predictions.extend(list(predicted))

    return predictions

"""
Training an image classifier
----------------------------

We will do the following steps in order:

1. Load the randomized FashionMNIST training, validation and test datasets using
   torchvision. Use torchvision.transforms to apply transforms on the
   dataset.
2. Define a Convolution Neural Network - BaseNet
3. Define a loss function and optimizer
4. Train the network on training data and check performance on val set.
   Plot train loss and validation accuracies.
5. Try the network on test data and create .npy file for submission to Gradescope"""

# <<TODO>>: Based on the val set performance, decide how many
# epochs are apt for your model.
# ---------
EPOCHS = 30
# ---------
IS_GPU = True
TEST_BS = 256
TOTAL_CLASSES = 10
TRAIN_BS = 64
PATH_TO_Fashion = "data/fashion/"
PATH_TO_Fashion_TEST = "data/fashion/"

"""1.**Loading FashionMNIST**

We will load the FashionMNIST dataset with builtin dataset loader from Torchvision.

"""

# # The output of torchvision datasets are PILImage images of range [0, 1].
# # Using transforms.ToTensor(), transform them to Tensors of normalized range
# # [-1, 1].


# # <<TODO#1>> Use transforms.Normalize() with the right parameters to
# # make the data well conditioned (zero mean, std dev=1) for improved training.
# # <<TODO#2>> Try using transforms.RandomCrop() and/or transforms.RandomHorizontalFlip()
# # to augment training data.
# # After your edits, make sure that test_transform should have the same data
# # normalization parameters as train_transform
# # You shouldn't have any data augmentation in test_transform (val or test data is never augmented).
# # ---------------------
# train_transform = transforms.Compose([
#     #transforms.RandomCrop(28, padding=2),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.2858,), (0.3204,))  # normalize to have zero mean and unit standard deviation
# ])

# test_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.2858,), (0.3204,))  # use the same normalization parameters as train_transform
# ])
# # ---------------------

# #DO NOT CHANGE any line below
# dataset = torchvision.datasets.FashionMNIST(root=PATH_TO_Fashion, train=True, download=True, transform=train_transform)
# print("dataset data shape: ", np.array(dataset.data).shape)
# print("dataset labels shape: ", np.array(dataset.targets).shape)

# # The 10 classes for FashionMNIST
# classes = dataset.classes
# # train dataset 55, 000 samples
# train_dataset = torch.utils.data.Subset(dataset, np.arange(len(dataset)-5000))
# # val dataset 5, 000 samples
# val_dataset = torch.utils.data.Subset(dataset, np.arange(len(dataset)-5000, len(dataset)))

# # check for Dataloader function: https://pytorch.org/docs/stable/data.html
# trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2, drop_last=True)  #DO NOT CHANGE
# valloader = torch.utils.data.DataLoader(val_dataset, batch_size=TEST_BS, shuffle=False, num_workers=2, drop_last=False)

# The output of torchvision datasets are PILImage images of range [0, 1].
# Using transforms.ToTensor(), transform them to Tensors of normalized range
# [-1, 1].


# <<TODO#1>> Use transforms.Normalize() with the right parameters to
# make the data well conditioned (zero mean, std dev=1) for improved training.
# <<TODO#2>> Try using transforms.RandomCrop() and/or transforms.RandomHorizontalFlip()
# to augment training data.
# After your edits, make sure that test_transform should have the same data
# normalization parameters as train_transform
# You shouldn't have any data augmentation in test_transform (val or test data is never augmented).
# ---------------------
train_transform = transforms.Compose(
    [transforms.ToTensor()])
test_transform = transforms.Compose(
    [transforms.ToTensor()])
# ---------------------

#DO NOT CHANGE any line below
dataset = torchvision.datasets.FashionMNIST(root=PATH_TO_Fashion, train=True, download=True, transform=train_transform)
print("dataset data shape: ", np.array(dataset.data).shape)
print("dataset labels shape: ", np.array(dataset.targets).shape)

# The 10 classes for FashionMNIST
classes = dataset.classes
# train dataset 55, 000 samples
train_dataset = torch.utils.data.Subset(dataset, np.arange(len(dataset)-5000))
# val dataset 5, 000 samples
val_dataset = torch.utils.data.Subset(dataset, np.arange(len(dataset)-5000, len(dataset)))

# check for Dataloader function: https://pytorch.org/docs/stable/data.html
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BS, shuffle=True, num_workers=2, drop_last=True)  #DO NOT CHANGE
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=TEST_BS, shuffle=False, num_workers=2, drop_last=False)

# mean = 0.
# std = 0.
# num_samples = 0.
# for data, _ in trainloader:
#     batch_size = data.size(0)
#     data = data.view(batch_size, data.size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     num_samples += batch_size

# mean /= num_samples
# std /= num_samples

# print("Mean: ", mean)
# print("Std: ", std)

# mean = 0.
# std = 0.
# num_samples = 0.
# for data, _ in valloader:
#     batch_size = data.size(0)
#     data = data.view(batch_size, data.size(1), -1)
#     mean += data.mean(2).sum(0)
#     std += data.std(2).sum(0)
#     num_samples += batch_size

# mean /= num_samples
# std /= num_samples

# print("Mean: ", mean)
# print("Std: ", std)

# Commented out IPython magic to ensure Python compatibility.
#Load secret test set
#DO NOT CHANGE any line below
if not os.path.isfile(os.path.join("/content/gdrive/My Drive/Colab Notebooks/CS_543_MP4", PATH_TO_Fashion_TEST, "fashion_secret_test_data.npy")):
#   %pip install -U gdown
  import gdown
  url = "https://drive.google.com/uc?id=130ssD7hrrDLqQHKVD3luD0tXTedWjC8j"
  output = os.path.join(PATH_TO_Fashion_TEST, "fashion_secret_test_data.npy")
  gdown.download(url, output, quiet=False)

test_img = np.load(os.path.join(PATH_TO_Fashion_TEST, "fashion_secret_test_data.npy"))
test_dataset = FashionMNISTTest(PATH_TO_Fashion_TEST, transform=test_transform)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BS, shuffle=False, num_workers=2, drop_last=False) #DO NOT CHANGE

"""2.**Visualize FashionMNIST**

We will visualize some random images from the FashionMNIST dataset.
"""

# Let us show some of the training images, for fun.
# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images[:16]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(16)))

"""3.**Define a Convolution Neural Network**

Implement the BaseNet exactly. BaseNet consists of two convolutional modules (conv-relu-maxpool) and two linear layers. The precise architecture is defined below:

| Layer No.   | Layer Type  | Kernel Size | Input Dim   | Output Dim  | Input Channels | Output Channels |
    | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- | ----------- |
    | 1 | conv2d | 5 | 28 | 24 | 1 | 6 |
         | 2 | relu | - | 24 | 24 | 6 | 6 |
         | 3 | maxpool2d | 2 | 24 | 12 | 6 | 6 |
         | 4 | conv2d | 5 | 12 | 8 | 6 | 12 |
         | 5 | relu | - | 8 | 8 | 12 | 12 |
         | 6 | maxpool2d | 2 | 8 | 4 | 12 | 12 |
         | 7 | linear | - | 1 | 1 | 192 | 12 |
         | 8 | relu | - | 1 | 1 | 12 | 12 |
         | 9 | linear | - | 1 | 1 | 12 | 10 |
"""

import torch.nn as nn
import torch.nn.functional as F

class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=12)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=12, out_features=10)


    def forward(self, x):
        # TODO: define your model here
        x = self.relu1(self.conv1(x)) # Size([bs, 6, 24, 24])
        x = self.pool1(x)             # Size([bs, 6, 12, 12])

        # Second convolutional layer
        x = self.relu2(self.conv2(x)) # Size([bs, 12, 8, 8])
        x = self.pool2(x)             # Size([bs, 12, 4, 4])

        # Flatten for fully connected layers
        x = x.view(-1, 12 * 4 * 4)

        # Fully connected layers
        x = self.fc1(x)               # Size([bs, 192])
        x = self.relu3(x)             # size([bs, 12])
        x = self.fc2(x)               #size([bs, 10])
        return x

import torch.nn as nn
import torch.nn.functional as F

class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(6)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(12)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=96)
        self.bn3 = nn.BatchNorm1d(12)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=96, out_features=48)
        self.fc3 = nn.Linear(48, 24)
        self.fc4 = nn.Linear(24, 10)
        self.relu4 = nn.ReLU()
        self.relu5 = nn.ReLU()
        self.drop1 =nn.Dropout(0.20)
        self.drop2 = nn.Dropout(0.20)

    def forward(self, x):
        # TODO: define your model here
        x = self.relu1(self.bn1(self.conv1(x))) # Size([bs, 6, 24, 24])
        x = self.pool1(x)             # Size([bs, 6, 12, 12])
        x = self.drop1(x)
        # Second convolutional layer
        x = self.relu2(self.bn2(self.conv2(x))) # Size([bs, 12, 8, 8])
        x = self.pool2(x)             # Size([bs, 12, 4, 4])

        # Flatten for fully connected layers
        x = x.view(-1, 12 * 4 * 4)

        # Fully connected layers
        x = self.fc1(x)               # Size([bs, 192])
        x = self.relu3(x)             # size([bs, 12])
        x = self.drop2(x)
        x = self.fc2(x)
        x = self.relu4(x)
        x = self.fc3(x)
        x = self.relu5(x)
        x = self.fc4(x)               #size([bs, 10])
        return x

import torch.nn as nn
import torch.nn.functional as F

class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)  # chnl-in, out, kernel
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(1024, 512)   # [64*4*4, x]
        self.bn3 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn4 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn5 = nn.BatchNorm1d(128)
        #self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(128, 10)      # 10 classes
        self.pool1 =nn.MaxPool2d(2, 2)   # kernel, stride
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop1 =nn.Dropout(0.20)
        self.drop2 = nn.Dropout(0.20)
        self.relu = nn.ReLU()

    def forward(self, x):
        # convolution phase
        z = self.relu(self.bn1(self.conv1(x)))  # Size([bs, 32, 24, 24])
        z = self.pool1(z)                       # Size([bs, 32, 12, 12])
        z = self.drop1(z)
        z = self.relu(self.bn2(self.conv2(z)))  # Size([bs, 64, 8, 8])
        z = self.pool2(z)                       # Size([bs, 64, 4, 4])

        # neural network phase
        z = z.reshape(-1, 1024)     # Size([bs, 1024])
        z = self.relu(self.bn3(self.fc1(z)))     # Size([bs, 512])
        z = self.drop2(z)
        z = self.relu(self.bn4(self.fc2(z)))    # Size([bs, 256])
        z = self.relu(self.bn5(self.fc3(z)))
        #z = self.relu(self.fc4(z))
        z = self.fc5(z)
        return z

# Create an instance of the nn.module class defined above:
net = BaseNet()

# Test your BaseNet with some random input
dummy_input = torch.rand((1, 1, 28, 28))
output = net(dummy_input)
assert output.shape == torch.Size([1, 10])

# For training on GPU, we need to transfer net and data onto the GPU
# http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#training-on-gpu
if IS_GPU:
    net = net.cuda()
########################################################################
# We provide a basic network that you should understand, run and
# eventually improve
# <<TODO>> Add more conv layers
# <<TODO>> Add more fully connected (fc) layers
# <<TODO>> Add regularization layers like Batchnorm.
#          nn.BatchNorm2d after conv layers:
#          http://pytorch.org/docs/master/nn.html#batchnorm2d
#          nn.BatchNorm1d after fc layers:
#          http://pytorch.org/docs/master/nn.html#batchnorm1d
# This is a good resource for developing a CNN for classification:
# http://cs231n.github.io/convolutional-networks/#layers

# TODO: paste output in your report
print(net)

"""4.**Define a loss function and optimizer**"""

########################################################################
# Here we use Cross-Entropy loss and SGD with momentum.
# The CrossEntropyLoss criterion already includes softmax within its
# implementation. That's why we don't use a softmax in our model
# definition.

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
criterion = nn.CrossEntropyLoss()

# Tune the learning rate.
# See whether the momentum and weight decay is useful or not
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0)
#optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

"""5.**Train the model**"""

########################################################################
# We simply have to loop over our data iterator, and feed the inputs to the
# network and optimize. We evaluate the validation accuracy at each
# epoch and plot these values over the number of epochs
# Nothing to change here
# -----------------------------
plt.ioff()
fig = plt.figure()
train_loss_over_epochs = []
val_accuracy_over_epochs = []
import pdb
for epoch in tqdm(range(EPOCHS), total=EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        if IS_GPU:
            inputs = inputs.cuda()
            labels = labels.cuda()
            #pdb.set_trace()

        # zero the parameter gradients
        optimizer.zero_grad()
        net = net.cuda()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    scheduler.step()
    # Normalizing the loss by the total number of train batches
    running_loss/=len(trainloader)
    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss))

    # Scale of 0.0 to 100.0
    # Calculate validation set accuracy of the existing model
    val_accuracy, val_classwise_accuracy = \
        calculate_accuracy(valloader, net, IS_GPU)
    print('Accuracy of the network on the val images: %d %%' % (val_accuracy))

    # # Optionally print classwise accuracies
    # for c_i in range(TOTAL_CLASSES):
    #     print('Accuracy of %5s : %2d %%' % (
    #         classes[c_i], 100 * val_classwise_accuracy[c_i]))

    train_loss_over_epochs.append(running_loss)
    val_accuracy_over_epochs.append(val_accuracy)
# -----------------------------


# Plot train loss over epochs and val set accuracy over epochs
# Nothing to change here
# -------------
plt.subplot(2, 1, 1)
plt.ylabel('Train loss')
plt.plot(np.arange(EPOCHS), train_loss_over_epochs, 'k-')
plt.title('train loss and val accuracy')
plt.xticks(np.arange(EPOCHS, dtype=int))
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.arange(EPOCHS), val_accuracy_over_epochs, 'b-')
plt.ylabel('Val accuracy')
plt.xlabel('Epochs')
plt.xticks(np.arange(EPOCHS, dtype=int))
plt.grid(True)
plt.savefig("mp4_q1_plot.png")
plt.close(fig)
print('Finished Training')
# -------------

from torchsummary import summary
summary(net, input_size=(1, 28, 28))

"""6.**Evaluate the validation accuracy of your final model**"""

val_accuracy, val_classwise_accuracy = \
        calculate_accuracy(valloader, net, IS_GPU)
print('Accuracy of the final network on the val images: %.1f %%' % (val_accuracy))

# Optionally print classwise accuracies
for c_i in range(TOTAL_CLASSES):
    print('Accuracy of %5s : %.1f %%' % (
        classes[c_i], val_classwise_accuracy[c_i]))

"""7.**Visualize test set images**"""

# get some random training images
dataiter = iter(testloader)
images = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images[:16]))

"""8.**Evaluate your final model on the test set**

Submit `predictions.npy` to Gradescope to see your model's performance on the test set.
"""

# run inference on the test set
predictions = run_secret_test(testloader, net, IS_GPU)
# save predictions
predictions = np.asarray(predictions)
np.save("Q1_label_predictions.npy", predictions)

# import torch.nn as nn
# import torch.nn.functional as F

# class BaseNet(nn.Module):
#     def __init__(self):
#         super(BaseNet, self).__init__()

#         # First convolutional layer
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
#         self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         # Second convolutional layer
#         self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
#         self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

#         # Fully connected layers
#         self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=12)
#         self.relu3 = nn.ReLU()
#         self.fc2 = nn.Linear(in_features=12, out_features=10)


#     def forward(self, x):
#         # TODO: define your model here
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)

#         # Second convolutional layer
#         x = self.conv2(x)
#         x = self.relu2(x)
#         x = self.pool2(x)

#         # Flatten for fully connected layers
#         x = x.view(-1, 12 * 4 * 4)

#         # Fully connected layers
#         x = self.fc1(x)
#         x = self.relu3(x)
#         x = self.fc2(x)
#         return x

In [ ]:
# -*- coding: utf-8 -*-
"""CS543_MP4_part2_starter_code.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1cQWXsIuskrEvRPklplnXBtA5S7PMLh5H
"""

# Mounting your Google Drive is optional, and you could also simply copy and
# upload the data to your colab instance. This manula upload is also easy to do,
# but you will have to figure out how to do it.
from google.colab import drive
drive.mount('/content/gdrive/')

import os
if not os.path.exists("//content/gdrive/MyDrive/Datasets"):
    os.makedirs("/content/gdrive/My Drive/Colab Notebooks/CS_543_MP4")
os.chdir("//content/gdrive/MyDrive/Datasets")

# Commented out IPython magic to ensure Python compatibility.
# download dataset
if not os.path.exists("/content/gdrive/MyDrive/Datasets/data"):
#   %pip install -U gdown
  import gdown
  url = "https://drive.google.com/uc?id=1sdmNN6b3stiDCwyZbVsl5vS5VbPnKixa"
  gdown.download(url, quiet=False)
  !unzip -qqo data.zip
  !rm data.zip

!pwd

import glob
import os
import numpy as np
import random
import time
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import cv2
from PIL import Image
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils import data
from torchvision import models
from torchvision.transforms import ToTensor, Normalize

# global variable
device = torch.device("cuda:0")

# class SegmentationDataset(data.Dataset):
#     """
#     Data loader for the Segmentation Dataset. If data loading is a bottleneck,
#     you may want to optimize this in for faster training. Possibilities include
#     pre-loading all images and annotations into memory before training, so as
#     to limit delays due to disk reads.
#     """

#     def __init__(self, split="train", data_dir="data"):
#         assert (split in ["train", "val", "test"])
#         self.img_dir = os.path.join(data_dir, split)
#         self.classes = []
#         with open(os.path.join(data_dir, 'classes.txt'), 'r') as f:
#             for l in f:
#                 self.classes.append(l.rstrip())
#         self.n_classes = len(self.classes)
#         self.split = split
#         self.data = glob.glob(self.img_dir + '/*.jpg')
#         self.data = sorted([os.path.splitext(l)[0] for l in self.data])

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, index):
#         img = Image.open(self.data[index] + '.jpg')
#         if self.split == 'test':
#             gt = Image.new('RGB', img.size)
#         else:
#             gt = Image.open(self.data[index] + '.png')
#         img = ToTensor()(img)
#         img = Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
#         gt = np.asarray(gt)
#         gt = torch.from_numpy(np.array(gt)).long().unsqueeze(0)
#         return img, gt

import random
import torchvision.transforms.functional as F
from torchvision import transforms
class SegmentationDataset(data.Dataset):
    """
    Data loader for the Segmentation Dataset. If data loading is a bottleneck,
    you may want to optimize this in for faster training. Possibilities include
    pre-loading all images and annotations into memory before training, so as
    to limit delays due to disk reads.
    """

    def __init__(self, split="train", data_dir="data", input_size=(512, 512)):
        assert (split in ["train", "val", "test"])
        self.img_dir = os.path.join(data_dir, split)
        self.classes = []
        with open(os.path.join(data_dir, 'classes.txt'), 'r') as f:
            for l in f:
                self.classes.append(l.rstrip())
        self.n_classes = len(self.classes)
        self.split = split
        self.data = glob.glob(self.img_dir + '/*.jpg')
        self.data = sorted([os.path.splitext(l)[0] for l in self.data])
        self.input_size = input_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img = Image.open(self.data[index] + '.jpg')
        if self.split == 'test':
            gt = Image.new('RGB', img.size)
        else:
            gt = Image.open(self.data[index] + '.png')

        # Random horizontal flipping
        if random.random() > 0.5:
            img = F.hflip(img)
            gt = F.hflip(gt)

        # # Random cropping
        # i, j, h, w = transforms.RandomCrop.get_params(
        #     img, output_size=self.input_size)
        # img = F.crop(img, i, j, h, w)
        # gt = F.crop(gt, i, j, h, w)

        img = ToTensor()(img)
        img = Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
        gt = np.asarray(gt)
        gt = torch.from_numpy(np.array(gt)).long().unsqueeze(0)
        return img, gt

train_dataset = SegmentationDataset(split='train')

# vis the training set
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
fig, axs = plt.subplots(ncols=5, nrows=2, figsize=(22, 7))
for idx, ax_i in enumerate(axs.T):
    ax = ax_i[0]
    img, gt = train_dataset[idx]
    img = std * img.permute((1, 2, 0)).cpu().numpy() + mean
    gt = gt.squeeze().numpy()
    ax.imshow((img * 255).astype(np.uint8))
    ax.axis('off')
    ax = ax_i[1]
    ax.imshow(gt)
    ax.axis('off')
fig.tight_layout()
plt.savefig('vis_trainset.pdf', format='pdf', bbox_inches='tight')

import torch.nn as nn

##########
#TODO: design your own network here. The expectation is to write from scratch. But it's okay to get some inspiration
#from conference paper. We are providing a very simple network that does a single 1x1 convolution to prdict the class label.
##########
class MyModel(nn.Module):

    def __init__(self, n_classes): # modify model
        super(MyModel, self).__init__()
        self.res18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.encoder = nn.Sequential(*list(self.res18.children())[:-2])
        self.layers = nn.Sequential(nn.Conv2d(512, n_classes, 1, padding=0))
        # self.up=nn.Upsample(scale_factor=32, mode='bilinear', align_corners=True)



    def forward(self, img): # modify forward path
        #x = self.res18(img)
        x=self.encoder(img)
        #print(x.shape)
        x=self.layers(x)
        # x=self.up(x)
        x = torch.nn.functional.interpolate(x, (224, 224), mode='bilinear')
        return x

import torch
import torch.nn as nn
import torchvision.models as models

class MyModel(nn.Module):
    def __init__(self, n_classes):
        super(MyModel, self).__init__()

        # Load pre-trained ResNet18 model
        resnet18 = models.resnet18(pretrained=True)

        # Remove the fully connected and average pooling layers
        modules = list(resnet18.children())[:-2]

        # Add atrous convolutional layers for semantic segmentation
        modules += [
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, n_classes, kernel_size=1, stride=1),
            nn.Upsample(scale_factor=32, mode='bilinear', align_corners=False)
        ]

        # Combine the encoder and decoder into a single model
        self.resnet_segmentation = nn.Sequential(*modules)

    def forward(self, img):
        x = self.resnet_segmentation(img)
        return x

import torch
import torch.nn as nn
import torchvision.models as models

class MyModel(nn.Module):
    def __init__(self, n_classes):
        super(MyModel, self).__init__()

        # Load pre-trained ResNet18 model
        resnet18 = models.resnet18(pretrained=True)

        # Remove the fully connected and average pooling layers
        modules = list(resnet18.children())[:-2]

        # Add atrous convolutional layers for semantic segmentation
        modules += [
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, n_classes, kernel_size=1, stride=1),
            nn.Upsample(scale_factor=8, mode='bilinear', align_corners=False)
        ]

        # Combine the encoder and decoder into a single model
        self.resnet_segmentation = nn.Sequential(*modules)

    def forward(self, img):
        x = self.resnet_segmentation(img)
        return x

#[32, 3, 224, 224] image shape
#[32, 1, 224, 224] target shape
# [32, 6, 32, 32]  my output

import torch
import torch.nn as nn
import torchvision.models as models

class MyModel(nn.Module):
    def __init__(self, n_classes):
        super(MyModel, self).__init__()

        # Load pre-trained ResNet18 model
        resnet18 = models.resnet18(pretrained=True)

        # Remove the fully connected and average pooling layers
        modules = list(resnet18.children())[:-2]

        # Add atrous convolutional layers for semantic segmentation
        modules += [
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, n_classes, kernel_size=1, stride=1),
            nn.Upsample(scale_factor=4, mode='bilinear', align_corners=False)
        ]

        # Combine the encoder and decoder into a single model
        self.resnet_segmentation = nn.Sequential(*modules)

    def forward(self, img):
        x = self.resnet_segmentation(img)
        return x

#[32, 3, 224, 224] image shape
#[32, 1, 224, 224] target shape
# [32, 6, 32, 32]  my output

class MyModel(nn.Module):
    def __init__(self, n_classes):
        super(MyModel, self).__init__()

        # Load pre-trained ResNet18 model
        resnet18 = models.resnet18(pretrained=True)

        # Remove the fully connected and average pooling layers
        modules = list(resnet18.children())[:-2]

        # Add atrous convolutional layers for semantic segmentation
        modules += [
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, n_classes, kernel_size=1, stride=1),
            nn.Upsample(scale_factor=8, mode='bilinear', align_corners=False)
        ]

        # Combine the encoder and decoder into a single model
        self.resnet_segmentation = nn.Sequential(*modules)

    def forward(self, img):
        x = self.resnet_segmentation(img)
        return x

class MyModel(nn.Module):
    def __init__(self, n_classes):
        super(MyModel, self).__init__()

        # Load pre-trained ResNet18 model
        resnet18 = models.resnet18(pretrained=True)

        # Remove the fully connected and average pooling layers
        modules = list(resnet18.children())[:-2]

        # Add atrous convolutional layers for semantic segmentation
        modules += [
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=2, dilation=2),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, n_classes, kernel_size=1, stride=1),
            nn.Upsample(scale_factor=8, mode='bilinear', align_corners=False)
        ]

        # Combine the encoder and decoder into a single model
        self.resnet_segmentation = nn.Sequential(*modules)

    def forward(self, img):
        x = self.resnet_segmentation(img)
        return x

# ##########
# #TODO: define your loss function here, we provide the basic Cross Entropy Loss
# ##########
import torch.nn as nn

# class MyCriterion(nn.Module):
#     def __init__(self):
#         super(MyCriterion, self).__init__()
#         #self.n_classes = n_classes
#         self.criterion = nn.CrossEntropyLoss(ignore_index=255)

#     def forward(self, prediction, target):
#         #print("target",target.shape)
#         loss = self.criterion(prediction, target.squeeze())

#         return loss

class MyCriterion(nn.Module):
    def __init__(self, n_classes, class_weights=None):
        super(MyCriterion, self).__init__()
        self.n_classes = n_classes
        self.class_weights = class_weights
        self.criterion = nn.CrossEntropyLoss(ignore_index=255, weight=self.class_weights)

    def forward(self, prediction, target):
        # Calculate the class weights if not provided
        if self.class_weights is None:
            target_flat = target.view(-1)
            class_counts = torch.bincount(target_flat, minlength=self.n_classes)
            class_weights = 1.0 / class_counts.float()
            self.criterion.weight = class_weights.to(prediction.device)

        loss = self.criterion(prediction, target.squeeze())

        return loss

########################################################################
# No need to modify below
# Evaluate sementic segmentation
# 1. Average precision of all classes and the average
# 2. Mean IOU of all classes and the average


import numpy as np
from sklearn.metrics import confusion_matrix


def segmentation_eval(gts, preds, classes):
    """
    @param    gts               numpy.ndarray   ground truth labels
    @param    preds             numpy.ndarray   predicted labels
    @param    classes           string          class names
    """
    ious, counts = compute_confusion_matrix(gts, preds)
    aps = compute_ap(gts, preds)
    for i in range(len(classes)):
        print('{:>20s}: AP: {:0.2f}, IoU: {:0.2f}'.format(classes[i], aps[i], ious[i]))
    print('{:>20s}: AP: {:0.2f}, IoU: {:0.2f}'.format('mean', np.mean(aps), np.mean(ious)))
    return aps, ious


def compute_ap(gts, preds):
    aps = []
    for i in range(preds.shape[1]):
        ap, prec, rec = calc_pr(gts == i, preds[:, i:i + 1, :, :])
        aps.append(ap)
    return aps


def calc_pr(gt, out, wt=None):
    gt = gt.astype(np.float64).reshape((-1, 1))
    out = out.astype(np.float64).reshape((-1, 1))

    tog = np.concatenate([gt, out], axis=1) * 1.
    ind = np.argsort(tog[:, 1], axis=0)[::-1]
    tog = tog[ind, :]
    cumsumsortgt = np.cumsum(tog[:, 0])
    cumsumsortwt = np.cumsum(tog[:, 0] - tog[:, 0] + 1)
    prec = cumsumsortgt / cumsumsortwt
    rec = cumsumsortgt / np.sum(tog[:, 0])
    ap = voc_ap(rec, prec)
    return ap, rec, prec


def voc_ap(rec, prec):
    rec = rec.reshape((-1, 1))
    prec = prec.reshape((-1, 1))
    z = np.zeros((1, 1))
    o = np.ones((1, 1))
    mrec = np.vstack((z, rec, o))
    mpre = np.vstack((z, prec, z))

    mpre = np.maximum.accumulate(mpre[::-1])[::-1]
    I = np.where(mrec[1:] != mrec[0:-1])[0] + 1
    ap = np.sum((mrec[I] - mrec[I - 1]) * mpre[I])
    return ap


def compute_confusion_matrix(gts, preds):
    preds_cls = np.argmax(preds, 1)
    gts = gts[:, 0, :, :]
    conf = confusion_matrix(gts.ravel(), preds_cls.ravel())
    inter = np.diag(conf)
    union = np.sum(conf, 0) + np.sum(conf, 1) - np.diag(conf)
    union = np.maximum(union, 1)
    return inter / union, conf


def val(model, val_dataloader, device):
    preds, gts = [], []

    # Put model in evaluation mode.
    model.eval()
    for i, batch in enumerate(val_dataloader):
        img, gt = batch
        img = img.to(device)
        gt = gt.to(device).long()
        pred = model(img)
        pred = torch.softmax(pred, 1)
        preds.append(pred.detach().cpu().numpy())
        gts.append(gt.detach().cpu().numpy())
    gts = np.concatenate(gts, 0)
    preds = np.concatenate(preds, 0)
    aps, ious = segmentation_eval(gts, preds, val_dataset.classes)

    # Put model back in training mode
    model.train()
    return preds

# TODO: implement your train loop here, we provide a very basic training loop
def train(model, criterion, optimizer, train_dataloader, epoch, **kwargs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        # Zero out gradient blobs in the optimizer
        optimizer.zero_grad()
        img, gt = batch

        # Move data to device for training
        img = img.to(device)
        gt = gt.to(device).long()

        # Get model predictions
        pred = model(img)
        loss = criterion(pred, gt)
        loss.backward()

        # Take a step to update network parameters.
        optimizer.step()

# TODO: Implement your training cycles, make sure you evaluate on validation
# dataset and compute evaluation metrics every so often.
# You may also want to save models that perform well.
# Tune your own optimizer and number of epochs, learning rate, etc
import torch.optim.lr_scheduler as lr_scheduler

model = MyModel(n_classes=6).to(device)
criterion = MyCriterion(n_classes=6).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train_dataloader = data.DataLoader(train_dataset, batch_size=32,
                                    shuffle=True, num_workers=2,
                                    drop_last=True)

val_dataset = SegmentationDataset(split="val")
val_dataloader = data.DataLoader(val_dataset, batch_size=1,
                                 shuffle=False, num_workers=0,
                                 drop_last=False)
num_epochs = 40
for epoch in tqdm(range(num_epochs)):
    train(model, criterion, optimizer, train_dataloader, epoch)
    # consider reducing learning rate
    #scheduler.step()
    # test results on validation set
    if epoch % 10 ==0 or epoch == num_epochs-1:
        print("epoch: {}, performance on validation set".format(epoch))
        preds = val(model, val_dataloader, device)

from torchsummary import summary
summary(model, input_size=(3, 224, 224))

# visualization pred on validation set against GT
# Feel free to modify for your custom visualization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
# vis input & pred on test set
fig, axs = plt.subplots(ncols=5, nrows=2, figsize=(22, 7))
for idx, ax_i in enumerate(axs.T):
    ax = ax_i[0]
    img, gt = val_dataset[idx]
    img = std * img.permute((1, 2, 0)).cpu().numpy() + mean
    ax.imshow((img * 255).astype(np.uint8))
    ax.axis('off')
    ax = ax_i[1]
    pred = np.argmax(preds[idx], 0)
    ax.imshow(pred)
    ax.axis('off')
fig.tight_layout()
plt.savefig('vis_valset.pdf', format='pdf', bbox_inches='tight')

########################################################################
# No need to modify below
# Generate predictions on test split
def predict(model, test_dataloader, device):
    preds = []
    # Put model in evaluation mode.
    model.eval()
    for i, batch in enumerate(test_dataloader):
        img, _ = batch
        img = img.to(device)
        pred = model(img)
        pred = torch.softmax(pred, 1)
        preds.append(pred.detach().cpu().numpy())
    preds = np.concatenate(preds, 0)
    # Put model back in training mode
    model.train()
    return preds

test_dataset = SegmentationDataset(split="test")
test_dataloader = data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False)
preds = predict(model, test_dataloader, device)

# visualization pred on test set
# Feel free to pick your favorite images for visualization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
# vis input & pred on test set
fig, axs = plt.subplots(ncols=5, nrows=2, figsize=(22, 7))
for idx, ax_i in enumerate(axs.T):
    ax = ax_i[0]
    img, _ = test_dataset[idx]
    img = std * img.permute((1, 2, 0)).cpu().numpy() + mean
    ax.imshow((img * 255).astype(np.uint8))
    ax.axis('off')
    ax = ax_i[1]
    pred = np.argmax(preds[idx], 0)
    ax.imshow(pred)
    ax.axis('off')
fig.tight_layout()
plt.savefig('vis_testset.pdf', format='pdf', bbox_inches='tight')

# save prediction, please upload to Gradescope
np.save('Q2_sseg_predictions', (preds*255).astype(np.uint8))

# import torch
# import torch.nn as nn
# import torchvision.models as models

# # Load pre-trained ResNet-18 model
# resnet = models.resnet18(pretrained=True)

# # Remove the classifier and global average pooling layers
# modules = list(resnet.children())[:-2]
# resnet = nn.Sequential(*modules)

# # Add upsampling layers for semantic segmentation
# num_classes = 2 # Example: Binary classification task
# resnet_decoder = nn.Sequential(
#     nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
#     nn.ReLU(inplace=True),
#     nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
#     nn.ReLU(inplace=True),
#     nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
#     nn.ReLU(inplace=True),
#     nn.ConvTranspose2d(64, num_classes, kernel_size=4, stride=2, padding=1),
#     nn.ReLU(inplace=True),
#     nn.Upsample(scale_factor=32, mode='bilinear', align_corners=False)
# )

# # Combine the encoder and decoder into a single model
# resnet_segmentation = nn.Sequential(resnet, resnet_decoder)

# import torch
# import torch.nn as nn
# import torchvision.models as models

# # Load pre-trained ResNet-18 model
# resnet = models.resnet18(pretrained=True)

# # Remove the classifier and global average pooling layers
# modules = list(resnet.children())[:-2]
# resnet = nn.Sequential(*modules)

# # Add atrous convolutional layers for semantic segmentation
# num_classes = 2 # Example: Binary classification task
# resnet_decoder = nn.Sequential(
#     nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1, dilation=2),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1, dilation=2),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1, dilation=2),
#     nn.ReLU(inplace=True),
#     nn.Conv2d(64, num_classes, kernel_size=1, stride=1),
#     nn.Upsample(scale_factor=32, mode='bilinear', align_corners=False)
# )

# # Combine the encoder and decoder into a single model
# resnet_segmentation = nn.Sequential(resnet, resnet_decoder)

# import torch
# import torch.nn as nn
# import torchvision.models as models

# class ResNet18Encoder(nn.Module):
#     def __init__(self, pretrained=True):
#         super(ResNet18Encoder, self).__init__()
#         self.encoder = models.resnet18(pretrained=pretrained)
#         # remove the last two layers: AvgPool and FC
#         self.encoder = nn.Sequential(*list(self.encoder.children())[:-2])

#     def forward(self, x):
#         return self.encoder(x)

# class AtrousConvolutionDecoder(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(AtrousConvolutionDecoder, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, dilation=1)
#         self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=2, dilation=2)
#         self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=4, dilation=4)
#         self.conv4 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=8, dilation=8)
#         self.conv5 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=16, dilation=16)

#     def forward(self, x):
#         x1 = self.conv1(x)
#         x2 = self.conv2(x)
#         x3 = self.conv3(x)
#         x4 = self.conv4(x)
#         x5 = self.conv5(x)
#         return x1 + x2 + x3 + x4 + x5

# class FCNDecoder(nn.Module):
#     def __init__(self, in_channels, out_channels, num_classes):
#         super(FCNDecoder, self).__init__()
#         self.deconv1 = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)
#         self.conv1 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
#         self.deconv2 = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=4, stride=2, padding=1)
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
#         self.deconv3 = nn.ConvTranspose2d(out_channels, out_channels, kernel_size=4, stride=2, padding=1)
#         self.conv3 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
#         self.classifier = nn.Conv2d(out_channels, num_classes, kernel_size=1)

#     def forward(self, x):
#         x = self.deconv1(x)
#         x = self.conv1(x)
#         x = self.deconv2(x)
#         x = self.conv2(x)
#         x = self.deconv3(x)
#         x = self.conv3(x)
#         return self.classifier(x)

# class ResNet18Segmentation(nn.Module):
#     def __init__(self, num_classes, use_atrous=False, use_fcn=False):
#         super(ResNet18Segmentation, self).__init__()
#         self.encoder = ResNet18Encoder()
#         if use_atrous:
#             self.decoder = AtrousConvolutionDecoder(512, 256)
#         elif use_fcn:
#             self.decoder = FCNDecoder(512, 256, num_classes)

# import torch
# import torch.nn as nn
# import torchvision.models as models
# class MyModel(nn.Module):
#   def __init__(self, n_classes): # modify model
#     super(MyModel, self).__init__()
# # Load pre-trained ResNet-18 model
#     resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
#     print(resnet)
#     # Remove the classifier and global average pooling layers
#     modules = list(resnet.children())[:-2]
#     #resnet = nn.Sequential(*modules)

#     # Add atrous convolutional layers for semantic segmentation
#     num_classes = 6 # Example: Binary classification task
#     resnet_decoder = nn.Sequential(
#         nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1, dilation=2),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1, dilation=2),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1, dilation=2),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(64, num_classes, kernel_size=1, stride=1),
#         nn.Upsample(scale_factor=32, mode='bilinear', align_corners=False)
#     )

#     # Combine the encoder and decoder into a single model
#     self.resnet_segmentation = nn.Sequential(modules, resnet_decoder)
#   def forward(self, img):
#     x=self.resnet_segmentation(img)
#     return x